<a href="https://colab.research.google.com/github/Jixson1/BuchmeyerAssignment1/blob/main/Notebook2_Buchmeyer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook #2: K Nearest Neighbors and Normalization
**Name: Jackson Buchmeyer**


## Load in the data:
You'll notice that there are a lot of columns here. We're going to work with a subset of these columns, not the entire dataset.

In [1]:
import pandas
import numpy
from google.colab import drive 
drive.mount('/content/drive')


Mounted at /content/drive


In [76]:
data = pandas.read_csv('/content/drive/MyDrive/CS167Spring22/datasets/LifeExpectancyData.csv')
data.head()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62.0,0.01,71.279624,65.0,1154.0,19.1,83.0,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64.0,0.01,73.523582,62.0,492.0,18.6,86.0,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66.0,0.01,73.219243,64.0,430.0,18.1,89.0,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69.0,0.01,78.184215,67.0,2787.0,17.6,93.0,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71.0,0.01,7.097109,68.0,3013.0,17.2,97.0,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


## Exercise #1: 
**Work with the right subset**: 

You're not going to work with the whole data set, just the 'Status', 'Life expectancy' and 3-5 additional features of your choice. To start, make this subset of the original data. Also, make sure your subset contains only countries with the 'Status' feature as **Developing**.

*hint: your initial subset should have between 2,000 and 2,500 rows.*

In [77]:
data.shape

(2938, 22)

In [78]:
subset1 = data[data['Status'] == 'Developing'][['Status', 'Life expectancy', 'percentage expenditure', 'Measles', 'under-five deaths', 'Population']]
subset1.head()

,Status,Life expectancy,percentage expenditure,Measles,under-five deaths,Population
0,Developing,65.0,71.279624,1154.0,83.0,33736494.0
1,Developing,59.9,73.523582,492.0,86.0,327582.0
2,Developing,59.9,73.219243,430.0,89.0,31731688.0
3,Developing,59.5,78.184215,2787.0,93.0,3696958.0
4,Developing,59.2,7.097109,3013.0,97.0,2978599.0


In [79]:
subset1.shape

(2426, 6)

## Exercise #2:
**Check for null values**: If you have a null target value ('Life expectancy'), you will need to throw that example out. 

Replace each of the null values within your predictors with the average of the column.

*hint: Useful functions here are `isna()`, `any()`, `fillna()`, `value_counts()` and `dropna()`. If you're not sure what these do, look them up in the [pandas documentation](https://pandas.pydata.org/docs/).*

After you have eliminated null values, use `.isna().any()` to verify that your fields no longer contain any null values (all of the values should be False). 

In [80]:
clean_subset = subset1.dropna(subset = ['Life expectancy']).copy()
clean_subset.shape

(2400, 6)

In [81]:
clean_subset['Measles'].fillna(clean_subset['Measles'].mean(), inplace=True)
clean_subset['under-five deaths'].fillna(clean_subset['under-five deaths'].mean(), inplace=True)
clean_subset['percentage expenditure'].fillna(clean_subset['percentage expenditure'].mean(), inplace=True)
clean_subset['Population'].fillna(clean_subset['Population'].mean(), inplace=True)

clean_subset.isna().any()

Status                    False
Life expectancy           False
percentage expenditure    False
Measles                   False
under-five deaths         False
Population                False
dtype: bool

## Exercise #3
Write up a k-nearest-neighbors function like the one you made for the iris data set in class. It should be able to make life expectancy predictions for new countries. You should also be able to specify what you want to use as k.

You will need to incorporate the predictor features that you have chosen to be a part of your subset in your knn function.

Pay special attention to whether this problem is a classification or a regression, and how the answer to that question may change the output of your model. 

In [83]:
# knn
def knn(country, data, k):
  data_copy = data.copy()
  data_copy['distance_to_new'] = numpy.sqrt(
    (country['percentage expenditure']-data['percentage expenditure'])**2
    +(country['Measles']-data['Measles'])**2
    +(country['under-five deaths']-data['under-five deaths'])**2
    +(country['Population']-data['Population'])**2)
  
  sorted_data = data_copy.sort_values(['distance_to_new'])
  # regression so calculate mean() of closest samples 
  return sorted_data.iloc[0:k]['Life expectancy'].mean()

## Exercise #4
Demonstrate that your function works by making up some new values for hypothetical countries and using your function to display the predicted life expectancy for that country.


In [84]:
Jacksonia = {
    'percentage expenditure': 50,
    'Measles': 2000,
    'under-five deaths': 500,
    'Population': 12345
}

Urnessia = {
    'percentage expenditure': 20,
    'Measles': 50000,
    'under-five deaths': 0,
    'Population': 2000000
}

# k = 100
print('Jacksonia Life Expectancy: ', knn(Jacksonia, clean_subset, 100))
print('Urnessia Life Expectancy: ', knn(Urnessia, clean_subset, 100))

Jacksonia Life Expectancy:  69.82100000000003
Urnessia Life Expectancy:  61.39200000000002


## Exercise #5

Make a copy of the data and normalize the training data using Z-score.

*hint: `.copy()` will be useful here.* 

Predict the life expectancy with some input parameters using your k-nearest-neighbors function with both the normalized and non-normalized training data. Compare your results. Use a markup cell to describe and explain the differences in a few sentences.

In [85]:
# normalizing data

norm_subset = clean_subset.copy()

expenditure_mean = clean_subset['percentage expenditure'].mean()
expenditure_std = clean_subset['percentage expenditure'].std()

measles_mean = clean_subset['Measles'].mean()
measles_std = clean_subset['Measles'].std()

deaths_mean = clean_subset['under-five deaths'].mean()
deaths_std = clean_subset['under-five deaths'].std()

population_mean = clean_subset['Population'].mean()
population_std = clean_subset['Population'].std()

In [86]:
# normalizing countries and running them through the knn function again
norm_Jacksonia = {
    'percentage expenditure': (Jacksonia['percentage expenditure'] - expenditure_mean) / expenditure_std,
    'Measles': (Jacksonia['Measles'] - measles_mean) / measles_std,
    'under-five deaths': (Jacksonia['under-five deaths'] - deaths_mean) / deaths_std,
    'Population': (Jacksonia['Population'] - population_mean) / population_std
}

norm_Urnessia = {
    'percentage expenditure': (Urnessia['percentage expenditure'] - expenditure_mean) / expenditure_std,
    'Measles': (Urnessia['Measles'] - measles_mean) / measles_std,
    'under-five deaths': (Urnessia['under-five deaths'] - deaths_mean) / deaths_std,
    'Population': (Urnessia['Population'] - population_mean) / population_std
}

# k = 100

# Normalized
print('Normalized Jacksonia Life Expectancy: ', knn(norm_Jacksonia, norm_subset, 500))
print('Normalized Urnessia Life Expectancy: ', knn(norm_Urnessia, norm_subset, 500))

print()

# Non-normalized
print('Non-normalized Jacksonia Life Expectancy: ', knn(Jacksonia, clean_subset, 100))
print('Non-normalized Urnessia Life Expectancy: ', knn(Urnessia, clean_subset, 100))

Normalized Jacksonia Life Expectancy:  67.88840000000002
Normalized Urnessia Life Expectancy:  67.88840000000002

Non-normalized Jacksonia Life Expectancy:  69.82100000000003
Non-normalized Urnessia Life Expectancy:  61.39200000000002


##**Observations**
- When running the **non-normalized** test countries through the regression knn
function, there **is** a noticable difference in the predicted life expectancy.
- When running the **normalized** test countries through the regression knn function, the predictions are **the same**. Even after changing the values drastically, the predicted life expectancy really did not change.
- After messing around with the k-values and the values of the test countries, I kept coming to essentially the same outcome.
- My theory is that certain predictor variables have larger effects on the predicted life expectancy than others Therefore normalizing the data makes all predictor variables have an equal effect on the predicted life expectancy, so changing them does not have as drastic an effect on the prediction than it would in a non-normalized k-nearest neighbor test.